<a href="https://colab.research.google.com/github/mgovindasamy/Machine-Learning-Examples/blob/master/JPY-tax-research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install download

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from download import download

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/mgovindasamy/Machine-Learning-Examples/master/Google_train_data.csv')
data.head()

These Python technical indicators are popularly used in the markets to study the price movement, so we will be adding the following features in our research:
*   Commodity Channel Index
*   Ease of Movement
*   Moving Average
*   Rate of Change
*   Bollinger Bands
*   Force Index


Formula for the Commodity Channel Index
The formula for CCI is given by:

CCI = (Typical price – MA of Typical price) / (0.015 * mean deviation of Typical price)
0.015 is Lambert's constant.

Estimation
To calculate the EMV we first calculate the distance moved. It is given by:

Distance moved = ((Current High + Current Low)/2 - (Prior High + Prior Low)/2)


To compute the 20-day EMA, we first compute the very first EMA value using a simple moving average. Then we calculate the multiplier, and thereafter to compute the second EMA value we use the multiplier and the previous day EMA. This formula is used to compute the subsequent EMA values.

SMA: 20 period sum / 20
Multiplier: (2 / (Time periods + 1)) = (2 / (20 + 1)) = 9.52%
EMA: {Close price - EMA(previous day)} x multiplier + EMA(previous day).

Estimation
ROC = [(Close price today - Close price “n” day’s ago) / Close price “n” day’s ago))]

Bollinger bands expand and contract based on the volatility. During a period of rising volatility, the bands widen, and they contract as the volatility decreases. Prices are considered to be relatively high when they move above the upper band and relatively low when they go below the lower band.

Estimation
To create the bands, we first compute the SMA and then use this to compute the bands values.

Middle Band = 20-day simple moving average (SMA)
Upper Band = 20-day SMA + (2 x 20-day standard deviation of price)
Lower Band = 20-day SMA - (2 x 20-day standard deviation of price)

Estimation
Example: Computing Force index(1) and Force index(15) period.

The Force index(1) = {Close (current period) - Close (prior period)} x Current period Volume
The Force Index for the 15-day period is an exponential moving average of the 1-period Force Index.



In [4]:
# Commodity Channel Index 
def CCI(df, ndays): 
  df['TP'] = (data['High'] + data['Low'] + data['Close']) / 3 
  df['sma'] = df['TP'].rolling(ndays).mean()
  df['mad'] = df['TP'].rolling(ndays).apply(lambda x: pd.Series(x).mad())
  df['CCI'] = (df['TP'] - df['sma']) / (0.015 * df['mad']) 
  return df

In [5]:
# Ease of Movement 
def EVM(df, ndays): 
 dm = ((df['High'] + df['Low'])/2) - ((df['High'].shift(1) + df['Low'].shift(1))/2)
 br = (df['Volume'] / 100000000) / ((df['High'] - df['Low']))
 EVM = dm / br 
 EVM_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM') 
 df = df.join(EVM_MA) 
 return df 

In [6]:
# Simple Moving Average 
def SMA(df, ndays): 
 SMA = pd.Series(df['Close'].rolling(ndays).mean(), name = 'SMA') 
 df = df.join(SMA) 
 return df

In [7]:
# Exponentially-weighted Moving Average 
def EWMA(df, ndays): 
 EMA = pd.Series(df['Close'].ewm(span = ndays, min_periods = ndays - 1).mean(), 
                 name = 'EWMA_' + str(ndays)) 
 df = df.join(EMA) 
 return df

In [8]:
# Rate of Change (ROC)
def ROC(df,n):
 N = df['Close'].diff(n)
 D = df['Close'].shift(n)
 ROC = pd.Series(N/D,name='Rate of Change')
 df = df.join(ROC)
 return df 

In [10]:
# Compute the Bollinger Bands 
def BBANDS(df, n):
    MA = df.Close.rolling(n).mean()
    SD = df.Close.rolling(n).std()
    df['UpperBB'] = MA + (2 * SD) 
    df['LowerBB'] = MA - (2 * SD)
    return data

In [11]:
# Force Index 
def ForceIndex(df, ndays): 
    FI = pd.Series(df['Close'].diff(ndays) * df['Volume'], name = 'ForceIndex') 
    df = df.join(FI) 
    return df